In [ ]:
import cvxpy as cp
import pandas as pd
import numpy as np
import pvlib
import time
import matplotlib.pyplot as plt

import pvlib
from pvlib.pvsystem import PVSystem, Array, FixedMount
from pvlib.location import Location
from pvlib.modelchain import ModelChain

%load_ext autoreload
%autoreload 2

In [ ]:
# Note: This only requests full years of data, and only requests in UTC.  
# To get a full local-tz set of continuous hours regardless of leap years we need a year on either side.
latitude, longitude = 37.8, -122.4
TIMEZONE = 'US/Pacific'
start, end = '2019', '2021'  # Choose around 2020 because 2020 is a leap year

pvgis_hourly = pvlib.iotools.get_pvgis_hourly(latitude, longitude,
                                         start=start, end=end,
                                         surface_tilt=latitude, surface_azimuth=180,
                                         pvcalculation=True, peakpower=1.0,
                                         loss=7.75, trackingtype=0,
                                        )
weather_data = pvgis_hourly[0]
solar_ac_estimate = weather_data['P'].rename('solar') / 1000.0 # convert to watts
solar_ac_estimate = solar_ac_estimate.tz_convert(TIMEZONE)
solar_ac_estimate = solar_ac_estimate.resample('1h', closed='right').last()

In [ ]:
fname = 'data/pge-e78ff14c-c8c0-11ec-8cc7-0200170a3297-DailyUsageData/pge_electric_usage_interval_data_Service 1_1_2024-02-01_to_2025-01-31.csv'
sample_consumption = pd.read_csv(fname, parse_dates={'Datetime': ['DATE', 'START TIME']})
sample_consumption['Datetime'] = pd.DatetimeIndex(sample_consumption['Datetime']).tz_localize('US/Pacific', ambiguous='NaT', nonexistent='NaT')
sample_consumption = sample_consumption[sample_consumption['Datetime'].notnull()]
sample_consumption = sample_consumption.set_index('Datetime')
elec_usage = sample_consumption['USAGE (kWh)'].rename('load')

elec_end_date = elec_usage.index[-1]
if elec_usage.index[0] < (elec_end_date - pd.DateOffset(years=1)):  
    elec_usage = elec_usage.loc[elec_end_date - pd.DateOffset(years=1):elec_end_date]

In [ ]:
leap_day = None
for t in elec_usage.index:
    if (t.month==2) & (t.day==29):
        leap_day = t
        break

if leap_day:
    shift_by_yrs = 2020 - leap_day.year
elif elec_end_date.month == 12 & elec_end_date.day == 31:
    shift_by_yrs = 2019 - elec_end_date.year
elif elec_end_date.month > 2:
    shift_by_yrs = 2021 - elec_end_date.year
else:
    shift_by_yrs = 2019 - elec_end_date.year

solar_ac_estimate.index = (solar_ac_estimate.index.tz_convert('UTC') - pd.DateOffset(years=shift_by_yrs)).tz_convert(TIMEZONE)
solar_ac_estimate = solar_ac_estimate.resample('1h', closed='right').last().ffill()  # Deal with any gaps related to shifted DST; thankfully these are in the night

site_data = pd.DataFrame(elec_usage).join(solar_ac_estimate, how='left')
site_data.plot(alpha=0.5)

In [ ]:
px_buy = pd.Series(0.4, index=site_data.index, name='px_buy')
px_buy.loc[px_buy.between_time('16:00', '21:00').index] = 0.52

px_sell = pd.Series(0.05, index=site_data.index, name='px_sell')
px_sell.loc[px_sell.between_time('15:00', '20:00').index] = 0.08

input_df = pd.concat([site_data, px_buy, px_sell], axis=1)
# input_df = input_df['2025-01-29':]

In [ ]:
rt_eff = 0.85
e_max = 13.5
p_max = 5
dt = 1.0 

oneway_eff = np.sqrt(rt_eff)
backup_reserve = 0.2
e_min = backup_reserve * e_max

n = input_df.shape[0]
E_0 = e_min

E_transition = np.hstack([np.eye(n), np.zeros(n).reshape(-1,1)])

P_batt_charge = cp.Variable(n)
P_batt_discharge = cp.Variable(n)
P_grid_buy = cp.Variable(n)
P_grid_sell = cp.Variable(n)
E = cp.Variable(n+1)

# Power flows are all AC, and are signed relative to the bus: injections to the bus are positive, withdrawals/exports from the bus are negative

constraints = [-p_max <= P_batt_charge,
               P_batt_charge <= 0,
               0 <= P_batt_discharge,
               P_batt_discharge <= p_max,
               0 <= P_grid_buy,
               P_grid_sell <= 0,
               e_min <= E,
               E <= e_max,
               E[1:] == E_transition @ E - (P_batt_charge * oneway_eff + P_batt_discharge / oneway_eff) * dt,
               P_batt_charge + P_batt_discharge + P_grid_buy + P_grid_sell - input_df['load'] + input_df['solar'] == 0,
               E[0] == E_0
              ]

obj = cp.Minimize(P_grid_sell @ input_df['px_sell'] + P_grid_buy @ input_df['px_buy'])

prob = cp.Problem(obj, constraints)

opt_start = time.time()
prob.solve()
print(f"Optimization done in {time.time() - opt_start :.3f} seconds")

res = pd.DataFrame.from_dict({'P_batt': P_batt_charge.value + P_batt_discharge.value,
                    'P_grid': P_grid_buy.value + P_grid_sell.value,
                    'E': E[1:].value}).set_index(input_df.index)
output_data = pd.concat([input_df, res], axis=1)

_ = output_data.plot()

# ModelChain approach

The below is the more detailed, canonical approach to modeling power as outlined in the `pvlib` tutorials.

In [ ]:
# latitude, longitude, name, altitude, timezone
STARTYEAR, ENDYEAR = 2023, 2023

coordinates = [
    (32.2, -111.0, 'Tucson', 700, 'Etc/GMT+7'),
    (35.1, -106.6, 'Albuquerque', 1500, 'Etc/GMT+7'),
    (37.8, -122.4, 'San Francisco', 10, 'Etc/GMT+8'),
    (52.5, 13.4, 'Berlin', 34, 'Etc/GMT-1'),
]
location = coordinates[2]
latitude, longitude, name, altitude, timezone = location

sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')
sapm_inverters = pvlib.pvsystem.retrieve_sam('cecinverter')
module = sandia_modules['Canadian_Solar_CS5P_220M___2009_']
inverter = sapm_inverters['ABB__MICRO_0_25_I_OUTD_US_208__208V_']
temperature_model_parameters = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']

inverter = sapm_inverters['ABB__MICRO_0_25_I_OUTD_US_208__208V_']
weather = pvlib.iotools.get_pvgis_tmy(latitude, longitude, startyear=STARTYEAR, endyear=ENDYEAR)[0]
weather.index.name = "utc_time"
weather = weather.sort_index()

location = Location(
    latitude,
    longitude,
    name=name,
    altitude=altitude,
    tz=timezone,
)
mount = FixedMount(surface_tilt=latitude, surface_azimuth=180)
array = Array(
    mount=mount,
    module_parameters=module,
    temperature_model_parameters=temperature_model_parameters,
)
system = PVSystem(arrays=[array], inverter_parameters=inverter)
mc = ModelChain(system, location)
mc.run_model(weather)
ac_power = mc.results.ac
ac_power.index = ac_power.index.tz_convert('US/Pacific')